In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import pygame
import os
import numpy as np
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv,VecMonitor,VecNormalize
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback,BaseCallback
from gymnasium.wrappers import FlattenObservation,TimeLimit,NormalizeObservation


In [87]:
from deform_rl.envs.Cable_reshape_env.environment import CableReshapeV2
# from deform_rl.envs.sim.utils.seed_manager import init_manager
SEG_NUM = 10

In [88]:
def make_env(seed=None,num_envs=1,render_moder=None):
    # env = CableReshapeV2(render_mode='human',seed=seed,seg_num=SEG_NUM,cable_length=300,scale_factor=800)
    # env = TimeLimit(env, max_episode_steps=1000)
    # envs = make_vec_env(env_id=CableReshapeV2,n_envs=num_envs,wrapper_class=VecNormalize)
    # envs = VecNormalize(envs)
    def dummy():
        return CableReshapeV2(render_mode=render_moder,seed=seed,seg_num=SEG_NUM,cable_length=300,scale_factor=800)

    # envs = DummyVecEnv([dummy]*num_envs)
    # envs = VecNormalize(VecMonitor((envs)))
    envs = make_vec_env(CableReshapeV2)

    return envs

save_dir = "./saved_models/cable_reshape"
log_dir = "./logs/cable_reshape"
os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [89]:
env = make_env()
# check_env(env, warn=True)


In [90]:
class LinearAgent:
    def __init__(self,ctrl_num):
        self.ctrl_num = ctrl_num
    def predict(self,obs,deterministic=True):
        print("OBS:", obs)
        rel_pos = obs[:self.ctrl_num*2]
        print("REL_POS: ", rel_pos)

        print(rel_pos.shape)
        rel_pos_reshaped = rel_pos.reshape(self.ctrl_num,2)
        lenghts = np.linalg.norm(rel_pos_reshaped,axis=1)
        max_len = np.max(lenghts)
        # print(lenghts[:,None].shape)
        # print(rel_pos_reshaped.shape)
        actions = rel_pos_reshaped/max_len
        # print(actions.shape)
        return actions.flatten()
        

In [91]:
#show linear agent actions
la = LinearAgent(SEG_NUM)
obs = env.reset()
print(obs.shape)
for i in range(1000):
    # print(la.predict(obs))
    obs, reward, done, info =env.step([la.predict(obs[0])])
    env.render(mode='human')
    if done:
        obs = env.reset()
        
    if pygame.event.get(pygame.QUIT):
        break
env.close()

(1, 20)
OBS: [ 234.66617     15.604594   135.65292    -34.169304    23.567837
  -41.89752    -79.53095      0.8144744 -164.35597     59.647755
    0.           0.          60.           0.         120.
    0.         180.           0.         240.           0.       ]
REL_POS:  [ 234.66617     15.604594   135.65292    -34.169304    23.567837
  -41.89752    -79.53095      0.8144744 -164.35597     59.647755
    0.           0.          60.           0.         120.
    0.         180.           0.         240.           0.       ]
(20,)
OBS: [ 234.66617     15.604594   135.65292    -34.169304    23.567837
  -41.89752    -79.53095      0.8144744 -164.35597     59.647755
    0.           0.          60.           0.         120.
    0.         180.           0.         240.           0.       ]
REL_POS:  [ 234.66617     15.604594   135.65292    -34.169304    23.567837
  -41.89752    -79.53095      0.8144744 -164.35597     59.647755
    0.           0.          60.           0.         120.

In [92]:
eval_env = make_env(num_envs=1)
env = make_env(num_envs=4)




eval_callback = EvalCallback(
    eval_env=eval_env,
    n_eval_episodes=15,
    eval_freq=10000,
    best_model_save_path=save_dir,
    verbose=1,
    render=False
)


In [93]:

model = PPO('MlpPolicy', env, device='cpu', verbose=0,tensorboard_log="./logs/cable_reshape/").learn(800000, tb_log_name="ppo_cable_reshapeV2",callback=eval_callback)

KeyboardInterrupt: 

In [ ]:
model.learn(800000,tb_log_name="ppo_cable_reshape",callback=eval_callback,reset_num_timesteps=False)

Eval num_timesteps=2920000, episode_reward=-831.40 +/- 782.80
Episode length: 744.53 +/- 364.26
Eval num_timesteps=2960000, episode_reward=-1117.38 +/- 675.98
Episode length: 881.07 +/- 304.30
Eval num_timesteps=3000000, episode_reward=-883.63 +/- 768.66
Episode length: 764.67 +/- 390.43
Eval num_timesteps=3040000, episode_reward=-1197.50 +/- 1035.83
Episode length: 829.87 +/- 342.06
Eval num_timesteps=3080000, episode_reward=-969.44 +/- 665.50
Episode length: 848.87 +/- 305.31
Eval num_timesteps=3120000, episode_reward=-707.14 +/- 914.95
Episode length: 657.07 +/- 421.41
New best mean reward!
Eval num_timesteps=3160000, episode_reward=-613.23 +/- 596.50
Episode length: 724.53 +/- 392.69
New best mean reward!
Eval num_timesteps=3200000, episode_reward=-796.20 +/- 940.14
Episode length: 638.27 +/- 398.16
Eval num_timesteps=3240000, episode_reward=-542.35 +/- 783.74
Episode length: 576.13 +/- 402.49
New best mean reward!
Eval num_timesteps=3280000, episode_reward=-773.68 +/- 902.49
Episo

In [ ]:
model.learn(1500000,tb_log_name="ppo_cable_reshapeV2",callback=eval_callback,reset_num_timesteps=False)

Eval num_timesteps=2040000, episode_reward=-1.54 +/- 2.01
Episode length: 554.93 +/- 388.02
Eval num_timesteps=2080000, episode_reward=-1.58 +/- 1.65
Episode length: 677.27 +/- 406.93
Eval num_timesteps=2120000, episode_reward=-1.23 +/- 2.14
Episode length: 549.13 +/- 381.81
Eval num_timesteps=2160000, episode_reward=-1.59 +/- 1.45
Episode length: 724.93 +/- 390.49
Eval num_timesteps=2200000, episode_reward=-0.68 +/- 1.77
Episode length: 447.13 +/- 396.23
New best mean reward!
Eval num_timesteps=2240000, episode_reward=-0.62 +/- 2.17
Episode length: 375.67 +/- 287.49
New best mean reward!
Eval num_timesteps=2280000, episode_reward=-0.99 +/- 1.79
Episode length: 503.87 +/- 391.37
Eval num_timesteps=2320000, episode_reward=-1.31 +/- 2.06
Episode length: 567.33 +/- 363.77
Eval num_timesteps=2360000, episode_reward=-0.75 +/- 1.29
Episode length: 551.13 +/- 388.79
Eval num_timesteps=2400000, episode_reward=-1.05 +/- 1.68
Episode length: 509.87 +/- 372.18
Eval num_timesteps=2440000, episode_

In [ ]:
# init_manager(15,15)
# random pick actions and visualize
tenv = VecNormalize(make_vec_env(make_env,n_envs=1))
print(tenv)
obs = tenv.reset()
# tenv = eval_env

# t_model = model
t_model = PPO.load(os.path.join(save_dir, "best_model.zip"),force_reset=True,device='cpu')
EP_CNT = 10
ep_cnt = 0
cnt = 0
rev_sum = 0
for i in range(10000):
    if cnt >= 1000:
        print("Killed by timeout")
        obs = tenv.reset()
        cnt = 0
    action,_ = t_model.predict(obs, deterministic=True)
    obs, reward, done, info = tenv.step(action)
    rev_sum += reward
    tenv.render()
    if done:
        obs,_ = tenv.reset()
        ep_cnt += 1
        print("Episode done: ", cnt, "Reward: ", rev_sum)
        cnt=0
        rev_sum = 0
        if ep_cnt >= EP_CNT:
            break
    if pygame.event.get(pygame.QUIT):
        break
    cnt +=1
tenv.close()